In [11]:
import pandas as pd
import numpy as np

In [1]:
cd /Users/davidroden/Desktop/Capstone

/Users/davidroden/Desktop/Capstone


In [74]:
# read in honeypot1 csv as a pandas df
df = pd.read_csv('honeypot1_1.csv', encoding='utf-8', index_col=None)

In [75]:
# strip IPs of whitespace 
df['src_ip']=[x.lstrip() for x in df['src_ip']]
df['src_ip']=[x.split('.') for x in df['src_ip']]
df['src_ip'] = [''.join(x) for x in df['src_ip']]
df.tail()

,Unnamed: 0,dest_ip,dest_port,details,session_id,software,src_ip,src_port,timestamp
3145148,3145148,128.143.31.40,0,login attempt [/root] failed,0,SSHService 'ssh-userauth' on HoneyPotSSHTransport,510140159,4018,2018-08-24 23:59:56.439101+00:00
3145149,3145149,128.143.31.40,0,'' failed auth 'password',0,-,0,0,2018-08-24 23:59:57.440624+00:00
3145150,3145150,128.143.31.40,0,unauthorized login:,0,-,0,0,2018-08-24 23:59:57.440863+00:00
3145151,3145151,128.143.31.40,0,connection lost,0,HoneyPotSSHTransport,510140159,4018,2018-08-24 23:59:57.538508+00:00
3145152,3145152,128.143.31.40,0,Connection lost after 1 seconds,0,HoneyPotSSHTransport,510140159,4018,2018-08-24 23:59:57.538762+00:00


In [76]:
# get list of login credentials attempted 
connection_password=[]
for i in df['details']:
    if 'login attempt' in i and ']' in i:
        temp=i.split('[')[1].split(']')[0]
        connection_password.append(temp)
len(connection_password)

28780

In [77]:
# get list of usernames and passwords to be added as columns to the dataframe
user = []
passw = []
for index, row in df.iterrows():
    if 'login attempt' in row['details'] and ']' in row['details']:
        temp=row['details'].split('[')[1].split(']')[0]
        temp2=temp.split('/')
        user.append(temp2[0])    
        if len(temp2)==2:
            passw.append(temp2[1])
        else:
            passw.append(' ')
    else:
        user.append('NaN')
        passw.append('NaN')
        

In [78]:
# add lists generated above as columns
df["username"] = user
df["password"] = passw
df.head()

,Unnamed: 0,dest_ip,dest_port,details,session_id,software,src_ip,src_port,timestamp,username,password
0,0,128.143.31.40,0,"Python Version 2.7.13 (default, Nov 24 2017, ...",0,-,0,0,2018-08-03 17:13:49.070316+00:00,NaN,NaN
1,1,128.143.31.40,0,Twisted Version 18.7.0,0,-,0,0,2018-08-03 17:13:49.070369+00:00,NaN,NaN
2,2,128.143.31.40,0,Loaded output engine: jsonlog,0,-,0,0,2018-08-03 17:13:49.073059+00:00,NaN,NaN
3,3,128.143.31.40,0,twistd 18.7.0 (/home/cowrie/cowrie/cowrie-env...,0,twisted.scripts._twistd_unix.UnixAppLogger#info,0,0,2018-08-03 17:13:49.074273+00:00,NaN,NaN
4,4,128.143.31.40,0,reactor class: twisted.internet.epollreactor....,0,twisted.scripts._twistd_unix.UnixAppLogger#info,0,0,2018-08-03 17:13:49.074452+00:00,NaN,NaN


In [79]:
# create lists of Mirai usernames and passwords 
# https://github.com/securing/mirai_credentials/blob/master/mirai_creds.txt
mirai_user = []
mirai_pass = []
with open("mirai_creds.txt") as mytxt:
    for line in mytxt:
        newline = line.rstrip("\n").split(":")
        mirai_user.append(newline[0])
        mirai_pass.append(newline[1])


In [80]:
# check to see if username and password match, if so, store index of row in dataframe
matches = []
for index, row in df.iterrows():
    if row['username'] in mirai_user:
        temp_index = mirai_user.index(row['username'])
        if row['password'] == mirai_pass[temp_index]:
            matches.append(index)
#        else:
#            matches.append(0)
#    else:
#        matches.append(0)
        

    

In [81]:
# how many rows had username:password combinations matching those of Mirai?
len(matches)

1176

In [82]:
# get the indexes of username/password attempted logins in dataframe 
attempts = []
for index, row in df.iterrows():
    if 'login attempt' in row['details'] and ']' in row['details']:
        attempts.append(index)

In [83]:
# ratio
len(matches)/len(attempts)

0.04086170952050035

In [93]:
# number of unqique ips with mirai username and password 
mirai_ips = pd.DataFrame(df['src_ip'][matches])
len(mirai_ips['src_ip'].unique())

505

In [86]:
# number of unique source ips in entire honeypot1
len(df['src_ip'].unique())

2623

In [94]:
# number of unique IPs that tried to login with username and password 
creds_ips = df['src_ip'][attempts]
len(creds_ips.unique())

2056

In [95]:
2056/2623

0.7838353030880671

In [96]:
505/2056

0.2456225680933852

In [99]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
df['details']

0           Python Version 2.7.13 (default, Nov 24 2017, 17:33:09) [GCC 6.3.0 20170516]                                                                                                                                                     
1           Twisted Version 18.7.0                                                                                                                                                                                                          
2           Loaded output engine: jsonlog                                                                                                                                                                                                   
3           twistd 18.7.0 (/home/cowrie/cowrie/cowrie-env/bin/python2 2.7.13) starting up.                                                                                                                                                  
4           reactor class: twisted.internet.epollrea

In [100]:
df

,Unnamed: 0,dest_ip,dest_port,details,session_id,software,src_ip,src_port,timestamp,username,password
0,0,128.143.31.40,0,"Python Version 2.7.13 (default, Nov 24 2017, 17:33:09) [GCC 6.3.0 20170516]",0,-,0,0,2018-08-03 17:13:49.070316+00:00,NaN,NaN
1,1,128.143.31.40,0,Twisted Version 18.7.0,0,-,0,0,2018-08-03 17:13:49.070369+00:00,NaN,NaN
2,2,128.143.31.40,0,Loaded output engine: jsonlog,0,-,0,0,2018-08-03 17:13:49.073059+00:00,NaN,NaN
3,3,128.143.31.40,0,twistd 18.7.0 (/home/cowrie/cowrie/cowrie-env/bin/python2 2.7.13) starting up.,0,twisted.scripts._twistd_unix.UnixAppLogger#info,0,0,2018-08-03 17:13:49.074273+00:00,NaN,NaN
4,4,128.143.31.40,0,reactor class: twisted.internet.epollreactor.EPollReactor.,0,twisted.scripts._twistd_unix.UnixAppLogger#info,0,0,2018-08-03 17:13:49.074452+00:00,NaN,NaN
5,5,128.143.31.40,0,Unhandled Error,0,-,0,0,2018-08-03 17:13:49.078660+00:00,NaN,NaN
6,6,128.143.31.40,0,"Received SIGTERM, shutting down.",0,-,0,0,2018-08-03 17:15:23.863409+00:00,NaN,NaN
7,7,128.143.31.40,0,Main loop terminated.,0,-,0,0,2018-08-03 17:15:23.864069+00:00,NaN,NaN
8,8,128.143.31.40,0,Server Shut Down.,0,twisted.scripts._twistd_unix.UnixAppLogger#info,0,0,2018-08-03 17:15:23.864345+00:00,NaN,NaN
9,9,128.143.31.40,0,"Python Version 2.7.13 (default, Nov 24 2017, 17:33:09) [GCC 6.3.0 20170516]",0,-,0,0,2018-08-03 17:15:40.109750+00:00,NaN,NaN
